# Prepare data to fit RSRM

- Andrew J. Graves, Cory Clayton, Joon Soh, Gabe Yohe
- 02/12/21

## Run necessary scripts for RSRM

In [3]:
%%capture output
# Run scripts for RSRM (double quotes work for Windows and Mac)
%run "rsrm_prep"
%run "rsrm"

import pickle

## Pre-process, feature engineer, and run RSRM

In [4]:
# Create an event dictionary
event_dict = {'rest': 1, 'left/both fists': 2, 'right/both feet': 3}

# Specify tasks to include
oc_fist = [3, 7, 11]
#im_oc_fist = [4, 8, 12]
#both_fists_feet = [5, 9, 13]
#im_both_fists_feet = [6, 10, 14]

subjs = 'all'

# Specify event-timing
tmin, tmax = -.2, 4

# Instantiate the data set (raw locations, number of subjects / tasks)
data = Dataset('motormovement_imagine')

# Build the montage
mont = mne.channels.make_standard_montage('standard_1005')

# Instantiate RSRM 
#rsrm = RSRM(n_iter=100, gamma=1.0, features=50, rand_seed=42)

# Read in the task data of interest
# You can also specify a subset of subjects with a list or int, similar to "tasks" syntax
data.preproc(event_dict=event_dict, baseline_start=tmin, stim_dur=tmax,
             montage=mont, subjects=subjs, tasks=oc_fist, eog_chan='Fp2')

# Specify time-window and compute frequency features (log-spaced)
#data.feature_engineer(step_size=0.2, freq_min=3, freq_max=45, num_freq_fams=12)

# Fit the RSRM
#rsrm.fit(data.feature_mats)

# Write data out
with open('data.pkl', 'wb') as output:  
    pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)

# Read data in
#with open('data.pkl', 'rb') as input:
    #data = pickle.load(input)

100% [..........................................................................] 2596896 / 2596896S001R03.edf
Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 529 samples (3.306 sec)

Bad channels detected: ['T8', 'TP8', 'T10']
EEG ch

In [ ]:
data.epoch_list[0].events[:, 2]

foo = []
for i in data.epoch_list:
    foo.append(i.events[:, 2])
               
for i in range(len(data.epoch_list)):
    print(np.alltrue(foo[0] == foo[i]))

In [5]:
# Trying out raw voltage SRM input
srm_in = []
for i in data.epoch_list:
    shp = i.get_data().shape
    srm_in.append(i.get_data().reshape((shp[0], shp[1]*shp[2])).T)
   

In [31]:
# Instantiate RSRM 
rsrm = RSRM(n_iter=10, gamma=2.0, features=63, rand_seed=42)

# Fit the RSRM
rsrm.fit(srm_in)

Iteration Number: 1;             Delta: 0.9995982795169456;             Objective function: 0.00040172048305444643
Iteration Number: 2;             Delta: 2.1877776777126895e-05;             Objective function: 0.00037984270627731953


RSRM(features=63, gamma=2.0, rand_seed=42)

In [ ]:
data.epoch_list[0].average().plot()

In [ ]:
data.epoch_list[0]['left'].plot_psd(picks='eeg');

In [ ]:
data.epoch_list[1]['left'].plot_psd_topomap();

In [ ]:
data.epoch_list[1]['right'].plot_psd_topomap();

In [ ]:
for idx, wts in enumerate(rsrm.w_):
    # Stack the time-windowed arrays along the 3rd axis
    if idx == 0:
        X = wts.T @ srm_in[idx]
    else:
        X = np.concatenate((X, wts.T @ srm_in[idx]), axis=1)
        
labs = data.epoch_list[0].events[:, 2]
y = np.repeat(labs, len(srm_in))

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

log_reg = LogisticRegressionCV(Cs=10, multi_class='multinomial')

X = rsrm.r_.T
labs = data.epoch_list[0].events[:, 2]
y = labs

X_train, X_test, y_train, y_test = train_test_split(X.T, y, train_size=.7)

log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=100)

#X = rsrm.w_[6].T @ data.feature_mats[6]
#X = data.feature_mats[0]
#X = arr
#y = labs
#X = rsrm.r_
rf.fit(X_train, y_train)
rf.score(X_test, y_test)